### 7.3 Pretrained Faster R-CNN Model
⚠️⚠️⚠️ *Please open this notebook in Google Colab* by click below link ⚠️⚠️⚠️<br><br>
<a href="https://colab.research.google.com/github/Muhammad-Yunus/Belajar-OpenCV-ObjectDetection/blob/main/Pertemuan%207/7.3%20pretrained_faster_rcnn.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a><br><br><br>
- Click `Connect` button in top right Google Colab notebook,<br>
<img src="https://github.com/Muhammad-Yunus/Belajar-OpenCV-ObjectDetection/blob/main/Pertemuan%207/resource/cl-connect-gpu.png?raw=1" width="250px">
- If connecting process completed, it will turn to something look like this<br>
<img src="https://github.com/Muhammad-Yunus/Belajar-OpenCV-ObjectDetection/blob/main/Pertemuan%207/resource/cl-connect-gpu-success.png?raw=1" width="250px">

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

import torch
from torchvision import transforms
from torchvision.models.detection import fasterrcnn_resnet50_fpn

- Load image Sample

In [ ]:
# Download an example image from repo
import urllib
url, filename = ("https://github.com/Muhammad-Yunus/Belajar-OpenCV-ObjectDetection/raw/main/Pertemuan%207/cow.jpg", "cow.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

In [ ]:
# show image using matplot lib
def imshow(image, isBGR=True):
    if isBGR :
      image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.imshow(image)
    plt.axis("off")

In [ ]:
# Load an example image and preprocess it
image_path = 'cow.jpg'  # Replace with your image path
image = cv2.imread(image_path)
H, W = 224, 224  # Resize image to this size
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = cv2.resize(image, (W, H))  # Resize to a suitable input size for ResNet

imshow(image, isBGR=False)

In [ ]:
# Convert image to tensor and normalize as required by ResNet
input_tensor = torch.tensor(image).float().permute(2, 0, 1).unsqueeze(0)
input_tensor = input_tensor / 255.0
input_tensor = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)(input_tensor)

- Load Pretrained Faster R-CNN with backbone ResNet-50

In [ ]:
# Load the Faster R-CNN model pre-trained on COCO dataset from Torch Vision
model = fasterrcnn_resnet50_fpn(pretrained=True)

# Set the model to evaluation mode
model.eval()

In [ ]:
# Perform inference
with torch.no_grad():
    predictions = model(input_tensor)

# Example: accessing the prediction results
print(predictions)  # Prints out the bounding boxes, labels, and scores
